<h1 align=center><font size = 5> Assignment week 3, part 1: Neighborhoods in Toronto City </font></h1>

In [28]:
!apt-get install python3-bs4
!pip install lxml

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [5]:
!easy_install html5lib

Searching for html5lib
Best match: html5lib 0.9999999
Adding html5lib 0.9999999 to easy-install.pth file

Using /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages
Processing dependencies for html5lib
Finished processing dependencies for html5lib


In [35]:
import numpy as np
import pandas as pd 
import requests
data_link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [36]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data_link,'lxml')
#print(soup.prettify()) #Print index page of the link

toronto_table = soup.find('table', {'class':'wikitable sortable'})
#toronto_table #I can print our requested table of the project
data_table = toronto_table.find_all('tr')

In [39]:
#Get raw data
table = []
for tr in data_table:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    table.append(row)
df=pd.DataFrame(table, columns=["PostalCode", "Borough", "Neighborhood"])
df.head(5)
#You can see raw data below, so we need too clean it

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


## Data processing

In [40]:
#Drop 'Not assigned' cells from Borough
df=df[df.Borough!='Not assigned'] 
df=df.drop(df.index[0])
df_new=df.reset_index(drop=True)
df_new['Neighborhood']=df_new.Neighborhood.str.replace('\n','')

#Group dublicated PostalCodes and get the sum of Neighborhood  
df_new=df_new.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(", ".join).reset_index() 
#Assign the Borough value where Neighborhood is 'Not assigned'
df_new.Neighborhood = np.where(df_new.Neighborhood=='Not assigned', df_new.Borough, df_new.Neighborhood)

df_new.head(100)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery, Humberlea"
98,M9N,York,Weston


Let's see I assinged Borough value to Neighborhood 'Not assigned' value in below code

In [41]:
df_new.iloc[2]

PostalCode                                    M1E
Borough                               Scarborough
Neighborhood    Guildwood, Morningside, West Hill
Name: 2, dtype: object

In [42]:
df_new.shape

(103, 3)

In [ ]:
Applied Data Science Capstone Week 3
